In [22]:
import pandas as pd
import numpy as np

listingsDF = pd.read_csv('listings.csv')

calendarDF = pd.read_csv('calendar.csv')

reviewsDF = pd.read_csv('reviews.csv')

In [26]:
#We clean reviews first

#We need to remove '(email hidden)', special symbols
reviewsDF = reviewsDF.loc[reviewsDF['reviewer_name'].str.contains(r'[^\x00-\x7F]+') == False]

reviewsDF = reviewsDF.loc[reviewsDF['reviewer_name'].str.contains('email hidden') == False]

#Then replace '()', '-'
reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('[()]', '')

reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('[-]', ' ')

#Then replace 'And' with '&'
reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('And', '&')

#Now we remove all rows with no reviews
reviewsDF = reviewsDF.dropna(axis=0, how='any')

#Convert date from object into datetime
reviewsDF.date = pd.to_datetime(reviewsDF.date)

#Check for row
#reviewsDF.loc[reviewsDF['reviewer_id'] == 12120141]

reviewsDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84283 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84283 non-null int64
id               84283 non-null int64
date             84283 non-null datetime64[ns]
reviewer_id      84283 non-null int64
reviewer_name    84283 non-null object
comments         84283 non-null object
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 4.5+ MB


In [8]:
#Now we clean listings

#Instead of dropping we select what we need
listingsDF = listingsDF[['id', 'name', 'summary', 'space', 'description', 
                         'host_id', 'host_name', 'latitude', 'longitude', 'property_type', 'price', 
                         'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']]

#We drop all rows with empty cells
listingsDF = listingsDF.dropna(axis=0, how='any')

#Remove the '$' from price
listingsDF.price = listingsDF.price.str.replace('[$]', '')

#Replace host name 'And' with '&'
listingsDF.price = listingsDF.price.str.replace('And', '&')

#Convert price from object to float
listingsDF.price = listingsDF.price.astype(float)

#listingsDF.head()
listingsDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2668 entries, 1 to 3814
Data columns (total 19 columns):
id                             2668 non-null int64
name                           2668 non-null object
summary                        2668 non-null object
space                          2668 non-null object
description                    2668 non-null object
host_id                        2668 non-null int64
host_name                      2668 non-null object
latitude                       2668 non-null float64
longitude                      2668 non-null float64
property_type                  2668 non-null object
price                          2668 non-null float64
number_of_reviews              2668 non-null int64
review_scores_rating           2668 non-null float64
review_scores_accuracy         2668 non-null float64
review_scores_cleanliness      2668 non-null float64
review_scores_checkin          2668 non-null float64
review_scores_communication    2668 non-null float64
revi